In [1]:
pip install tensorflow_addons


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import shutil

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
base_dir = 'Capstone'
train_dir = 'Capstone/train'
test_dir = 'Capstone/test'

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(128, 128),
    seed=2,
    color_mode="rgb",
    batch_size=20,
    shuffle=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,  # You should define `val_dir` appropriately
    image_size=(128, 128),
    seed=2,
    color_mode="rgb",
    batch_size=20,
    shuffle=True,
)


Found 4026 files belonging to 17 classes.
Found 1773 files belonging to 17 classes.


In [6]:
#val_ds = tf.keras.utils.image_dataset_from_directory(base_dir, image_size=(128,128), seed=2, color_mode="rgb", batch_size=8, validation_split=0.2, subset="validation")

In [7]:
class_info = train_ds.class_names

In [8]:
def normalize_image(x, y):
  normalization_layer = tf.keras.layers.Rescaling(1./255)(x)
  return normalization_layer, y

In [9]:
train_ds = train_ds.map(normalize_image)

# Transfer Learning

In [10]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(128, 128, 3),
                                                include_top=False,
                                                weights='imagenet')

In [11]:
# Adding new layers to the models
base_model.trainable = True

x = base_model.output
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# Ensure that the output shape is 4D before applying GlobalAveragePooling2D
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_info), activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)


In [12]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']        

In [13]:
#Model compile with Adam optimizer, and categorical_crossentropy for the loss
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

In [14]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if logs.get('accuracy') > 0.80 and logs.get('val_accuracy') > 0.80:
      print("\nReached desired accuracy, so stopped training!")
      self.model.stop_training = True

callback = myCallback()

In [15]:
#Start training the model, and save the history
history = model.fit(train_ds,
                    epochs=50,
                    validation_data=val_ds,
                    callbacks = [callback])

Epoch 1/50
202/202 [==============================] - 152s 694ms/step - loss: 1.8195 - accuracy: 0.3684 - val_loss: 2.8619 - val_accuracy: 0.1754
Epoch 2/50
202/202 [==============================] - 135s 668ms/step - loss: 0.1526 - accuracy: 0.9508 - val_loss: 4.4936 - val_accuracy: 0.1844
Epoch 3/50
202/202 [==============================] - 136s 666ms/step - loss: 0.0540 - accuracy: 0.9839 - val_loss: 4.0234 - val_accuracy: 0.2267
Epoch 4/50
202/202 [==============================] - 139s 687ms/step - loss: 0.0405 - accuracy: 0.9883 - val_loss: 6.5002 - val_accuracy: 0.1455
Epoch 5/50
202/202 [==============================] - 131s 647ms/step - loss: 0.0181 - accuracy: 0.9945 - val_loss: 6.0053 - val_accuracy: 0.1433
Epoch 6/50
202/202 [==============================] - 136s 671ms/step - loss: 0.0182 - accuracy: 0.9945 - val_loss: 5.2675 - val_accuracy: 0.1320
Epoch 7/50
202/202 [==============================] - 127s 626ms/step - loss: 0.0215 - accuracy: 0.9943 - val_loss: 9.2310 -

In [16]:
model_name = 'yoga model7.h5'

model.save(model_name)

c:\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
from google.colab import files
files.download(model_name)

ModuleNotFoundError: No module named 'google.colab'